# **4. 모델 훈련 연습문제**


### 1) 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?

확률적 경사 하강법. 매 반복에서 다뤄야 할 데이터가 매우 적기 때문에 한 번에 하나의 샘플을 처리하고, 알고리즘이 훨씬 빠르기 때문.



### 9) 릿지 회귀를 사용했을 때 훈련 오차와 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 α 를 증가시켜야할까요, 아니면 줄여야 할까요?

분산이 높을 경우에는 과적합되어 훈련 오차가 검증 오차보다 훨씬 더 크게 나타난다. 따라서 이 모델의 경우에는 높은 편향이 문제이고, 규제 하이퍼파라미터를 감소시켜야 한다. (alpha를 증가시킬수록 모델의 분산은 줄고 편향이 커짐)



### 10) 다음과 같이 사용해야 하는 이유는?

- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀

- 릿지 회귀 대신 라쏘 회귀

- 라쏘 회귀 대신 엘라스틱넷

1) 과대적합 방지

2) 설명변수의 개수를 줄여 복잡도를 낮춘다.

3) 훈련샘플수보다 많거나 특성 몇 개가 과하게 연관되어있을 때

# 12) 다음 사이트에 들어가서 연습문제 12번 일부를 필사하세요.

https://github.com/rickiepark/handson-ml2/blob/master/04_training_linear_models.ipynb

- 67번 셀 부터 80번 셀 까지 필사하시면 됩니다.

- 시간이 남는다면 그 이후 코드까지 필사하셔도 좋습니다.

- 수식이 설명되어있는 부분을 보고싶으면 그 부분을 그대로 복사하여 텍스트 창에 붙여넣기 한 다음 직접 실행시키시면 확인해볼 수 있습니다.

In [1]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename']

In [2]:
X = iris["data"][:, (2, 3)]  # 꽃잎 길이, 꽃잎 넓이
y = iris["target"]

In [4]:
import numpy as np
#모든 샘플에 편향을 추가한다.
X_with_bias= np.c_[np.ones([len(X),1]),X]

In [5]:
# 결과를 일정하게 유지하기 위해 랜덤 시드를 지정합니다:
np.random.seed(2042)

In [7]:
# 데이터 나누기
test_ratio= 0.2
validation_ratio=0.2
total_size=len(X_with_bias)

test_size= int(total_size*test_ratio)
validation_size= int(total_size*validation_ratio)
train_size= total_size-test_size- validation_size

In [8]:
rnd_indices = np.random.permutation(total_size)

In [9]:
X_train= X_with_bias[rnd_indices[:train_size]]
y_train=y[rnd_indices[:train_size]]
X_valid= X_with_bias[rnd_indices[train_size:-test_size]]
y_valid = y[rnd_indices[train_size:-test_size]]
X_test = X_with_bias[rnd_indices[-test_size:]]
y_test = y[rnd_indices[-test_size:]]

In [10]:
# 클래스 인덱스를 원-핫 벡터로 바꾸는 간단한 함수
def to_one_hot(y):
    n_classes=y.max()+1
    m=len(y)
    Y_one_hot= np.zeros((m,n_classes))
    Y_one_hot[np.arange(m),y]=1
    return Y_one_hot

In [11]:
y_train[:10]

array([0, 1, 2, 1, 1, 0, 1, 1, 1, 0])

In [12]:
to_one_hot(y_train[:10])

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [15]:
# 훈련 세트와 테스트 세트의 타깃 클래스 확률을 담은 행렬 만들기
Y_train_one_hot = to_one_hot(y_train)
Y_valid_one_hot= to_one_hot(y_valid)
Y_test_one_hot = to_one_hot(y_test)

In [16]:
# 소프트 맥스 함수
def softmax(logits):
    exps = np.exp(logits)
    exp_sums = np.sum(exps, axis=1, keepdims=True)
    return exps / exp_sums

In [20]:
# 입력과 출력의 갯수 정의
n_inputs= X_train.shape[1]
n_outputs= len(np.unique(y_train))

In [21]:
# 비용함수를 구현하자
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7

Theta =np.random.randn(n_inputs, n_outputs)

In [22]:
for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba=softmax(logits)
    if iteration%500 ==0:
        loss=-np.mean(np.sum(Y_train_one_hot * np.log(Y_proba+epsilon),axis=1))
        print(iteration, loss)
    error =Y_proba-Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error)
    Theta = Theta - eta * gradients

0 5.446205811872683
500 0.8350062641405652
1000 0.6878801447192402
1500 0.6012379137693313
2000 0.5444496861981872
2500 0.5038530181431525
3000 0.4729228972192248
3500 0.44824244188957774
4000 0.4278651093928792
4500 0.4106007142918712
5000 0.3956780375390374


#### soft-max 훈련 완료

In [24]:
Theta  # model 파라미터

array([[ 3.32094157, -0.6501102 , -2.99979416],
       [-1.1718465 ,  0.11706172,  0.10507543],
       [-0.70224261, -0.09527802,  1.4786383 ]])

In [26]:
logits=X_valid.dot(Theta)
Y_proba=softmax(logits)
y_predict = np.argmax(Y_proba, axis=1)
accuracy_score = np.mean(y_predict == y_valid)
accuracy_score   #정확도 확인

0.9666666666666667

#### L2규제 해보자

In [29]:
eta = 0.01
n_iterations = 5001
m = len(X_train)
epsilon = 1e-7
alpha= 0.1 #규제 파라미터

Theta = np.random.randn(n_inputs, n_outputs)

In [30]:
for iteration in range(n_iterations):
    logits = X_train.dot(Theta)
    Y_proba=softmax(logits)
    if iteration%500 ==0:
        xentropy_loss=-np.mean(np.sum(Y_train_one_hot * np.log(Y_proba+epsilon),axis=1))
        l2_loss= 1/2* np.sum(np.square(Theta[1:]))
        loss=xentropy_loss+alpha*l2_loss
        print(iteration, loss)
    error =Y_proba-Y_train_one_hot
    gradients = 1/m * X_train.T.dot(error) +np.r_[np.zeros([1,n_outputs]),alpha*Theta[1:]]
    Theta = Theta - eta * gradients

0 5.401014020496038
500 0.9057746237871946
1000 0.7546451358045129
1500 0.677898020634155
2000 0.633156395629338
2500 0.6040928899583043
3000 0.5836979468114086
3500 0.5685796152886732
4000 0.5569165937721949
4500 0.5476455924677358
5000 0.5401037070007071
